In [6]:
import numpy as np
import gym
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
np.bool8 = np.bool_


env = gym.make("Pong-v4")
H = 200  # 隐藏层维度
D = 80 * 80  # 输入维度 (80x80)

class PolicyNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(D, H)
        self.fc2 = nn.Linear(H, 1)
        self._initialize_weights()

    def _initialize_weights(self):
        nn.init.xavier_normal_(self.fc1.weight)
        nn.init.xavier_normal_(self.fc2.weight)
        self.fc1.bias.data.zero_()
        self.fc2.bias.data.zero_()

    def forward(self, x):
        h = F.relu(self.fc1(x))
        logit = self.fc2(h)
        return h, torch.sigmoid(logit)

model = PolicyNet()
optimizer = optim.RMSprop(model.parameters(), lr=1e-3, alpha=0.99, eps=1e-5)

def prepro(I):
    """预处理帧图像"""
    I = I[35:195]
    I = I[::2, ::2, 0]
    I[I == 144] = 0
    I[I == 109] = 0
    I[I != 0] = 1
    return I.astype(np.float32).ravel()

def discounted_rewards(rewards):
    discounted = np.zeros_like(rewards, dtype=np.float32)
    running_add = 0
    for t in reversed(range(len(rewards))):
        if rewards[t] != 0: running_add = 0
        running_add = running_add * 0.99 + rewards[t]
        discounted[t] = running_add
    return (discounted - discounted.mean()) / (discounted.std() + 1e-8)

# 训练循环
observation, _ = env.reset()
prev_x = None
running_reward = None
batch_size = 10

xs, hs, logps, rewards = [], [], [], []
episode_number = 0

while True:
    current_x = prepro(observation)
    x = current_x - prev_x if prev_x is not None else np.zeros(D)
    x_tensor = torch.FloatTensor(x)
    
    # 前向传播
    with torch.no_grad():
        h_tensor, prob = model(x_tensor)
    
    # 采样动作
    action = 2 if np.random.rand() > prob.item() else 3
    y = 1 if action == 3 else 0
    
    # 需要重新计算梯度跟踪的版本
    h_tensor.requires_grad_()
    _, prob = model(x_tensor)
    log_prob = y * torch.log(prob) + (1 - y) * torch.log(1 - prob)
    
    # 存储轨迹信息
    xs.append(x_tensor)
    hs.append(h_tensor)
    logps.append(log_prob)
    
    # 执行动作
    observation, reward, terminated, truncated, _ = env.step(action)
    rewards.append(reward)
    
    if terminated or truncated:
        episode_number += 1
        
        # 计算折扣回报
        dr_tensor = torch.FloatTensor(discounted_rewards(np.array(rewards)))
        
        # 计算损失
        logps_tensor = torch.stack(logps)
        loss = -torch.sum(logps_tensor * dr_tensor)
        
        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # 更新运行奖励
        reward_sum = sum(rewards)
        running_reward = reward_sum if running_reward is None else running_reward*0.99 + reward_sum*0.01
        print(f"Episode {episode_number} reward: {reward_sum}, running: {running_reward:.2f}")
        
        # 重置环境
        xs, hs, logps, rewards = [], [], [], []
        observation, _ = env.reset()
        prev_x = None

Episode 1 reward: -21.0, running: -21.00
Episode 2 reward: -21.0, running: -21.00
Episode 3 reward: -19.0, running: -20.98
Episode 4 reward: -21.0, running: -20.98
Episode 5 reward: -21.0, running: -20.98
Episode 6 reward: -21.0, running: -20.98
Episode 7 reward: -21.0, running: -20.98
Episode 8 reward: -21.0, running: -20.98
Episode 9 reward: -21.0, running: -20.98
Episode 10 reward: -21.0, running: -20.98
Episode 11 reward: -21.0, running: -20.98
Episode 12 reward: -20.0, running: -20.97
Episode 13 reward: -20.0, running: -20.96
Episode 14 reward: -21.0, running: -20.96
Episode 15 reward: -20.0, running: -20.95
Episode 16 reward: -20.0, running: -20.94
Episode 17 reward: -20.0, running: -20.93
Episode 18 reward: -20.0, running: -20.92
Episode 19 reward: -20.0, running: -20.92
Episode 20 reward: -19.0, running: -20.90
Episode 21 reward: -20.0, running: -20.89
Episode 22 reward: -21.0, running: -20.89
Episode 23 reward: -21.0, running: -20.89
Episode 24 reward: -20.0, running: -20.88
E

Episode 195 reward: -21.0, running: -20.51
Episode 196 reward: -20.0, running: -20.50
Episode 197 reward: -21.0, running: -20.51
Episode 198 reward: -21.0, running: -20.51
Episode 199 reward: -21.0, running: -20.52
Episode 200 reward: -21.0, running: -20.52
Episode 201 reward: -21.0, running: -20.53
Episode 202 reward: -21.0, running: -20.53
Episode 203 reward: -21.0, running: -20.53
Episode 204 reward: -21.0, running: -20.54
Episode 205 reward: -20.0, running: -20.53
Episode 206 reward: -21.0, running: -20.54
Episode 207 reward: -21.0, running: -20.54
Episode 208 reward: -21.0, running: -20.55
Episode 209 reward: -20.0, running: -20.54
Episode 210 reward: -21.0, running: -20.55
Episode 211 reward: -21.0, running: -20.55
Episode 212 reward: -18.0, running: -20.53
Episode 213 reward: -21.0, running: -20.53
Episode 214 reward: -19.0, running: -20.52
Episode 215 reward: -21.0, running: -20.52
Episode 216 reward: -19.0, running: -20.50
Episode 217 reward: -20.0, running: -20.50
Episode 218

Episode 386 reward: -21.0, running: -20.44
Episode 387 reward: -20.0, running: -20.43
Episode 388 reward: -21.0, running: -20.44
Episode 389 reward: -21.0, running: -20.44
Episode 390 reward: -20.0, running: -20.44
Episode 391 reward: -20.0, running: -20.44
Episode 392 reward: -20.0, running: -20.43
Episode 393 reward: -21.0, running: -20.44
Episode 394 reward: -21.0, running: -20.44
Episode 395 reward: -20.0, running: -20.44
Episode 396 reward: -20.0, running: -20.43
Episode 397 reward: -20.0, running: -20.43
Episode 398 reward: -20.0, running: -20.43
Episode 399 reward: -21.0, running: -20.43
Episode 400 reward: -21.0, running: -20.44
Episode 401 reward: -21.0, running: -20.44
Episode 402 reward: -21.0, running: -20.45
Episode 403 reward: -21.0, running: -20.45
Episode 404 reward: -21.0, running: -20.46
Episode 405 reward: -21.0, running: -20.46
Episode 406 reward: -20.0, running: -20.46
Episode 407 reward: -21.0, running: -20.46
Episode 408 reward: -20.0, running: -20.46
Episode 409

Episode 577 reward: -21.0, running: -20.54
Episode 578 reward: -21.0, running: -20.54
Episode 579 reward: -19.0, running: -20.53
Episode 580 reward: -20.0, running: -20.52
Episode 581 reward: -21.0, running: -20.52
Episode 582 reward: -21.0, running: -20.53
Episode 583 reward: -20.0, running: -20.52
Episode 584 reward: -21.0, running: -20.53
Episode 585 reward: -21.0, running: -20.53
Episode 586 reward: -20.0, running: -20.53
Episode 587 reward: -21.0, running: -20.53
Episode 588 reward: -21.0, running: -20.54
Episode 589 reward: -21.0, running: -20.54
Episode 590 reward: -21.0, running: -20.55
Episode 591 reward: -21.0, running: -20.55
Episode 592 reward: -21.0, running: -20.56
Episode 593 reward: -21.0, running: -20.56
Episode 594 reward: -19.0, running: -20.54
Episode 595 reward: -20.0, running: -20.54
Episode 596 reward: -21.0, running: -20.54
Episode 597 reward: -20.0, running: -20.54
Episode 598 reward: -21.0, running: -20.54
Episode 599 reward: -21.0, running: -20.55
Episode 600

Episode 768 reward: -21.0, running: -20.54
Episode 769 reward: -21.0, running: -20.54
Episode 770 reward: -21.0, running: -20.55
Episode 771 reward: -21.0, running: -20.55
Episode 772 reward: -21.0, running: -20.56
Episode 773 reward: -20.0, running: -20.55
Episode 774 reward: -18.0, running: -20.53
Episode 775 reward: -21.0, running: -20.53
Episode 776 reward: -21.0, running: -20.54
Episode 777 reward: -21.0, running: -20.54
Episode 778 reward: -21.0, running: -20.54
Episode 779 reward: -21.0, running: -20.55
Episode 780 reward: -20.0, running: -20.54
Episode 781 reward: -20.0, running: -20.54
Episode 782 reward: -21.0, running: -20.54
Episode 783 reward: -20.0, running: -20.54
Episode 784 reward: -21.0, running: -20.54
Episode 785 reward: -21.0, running: -20.55
Episode 786 reward: -21.0, running: -20.55
Episode 787 reward: -21.0, running: -20.56
Episode 788 reward: -18.0, running: -20.53
Episode 789 reward: -20.0, running: -20.52
Episode 790 reward: -20.0, running: -20.52
Episode 791

...

Episode 48551 reward: -21.0, running: -20.44
Episode 48552 reward: -20.0, running: -20.44
Episode 48553 reward: -21.0, running: -20.44
Episode 48554 reward: -20.0, running: -20.44
Episode 48555 reward: -21.0, running: -20.45
Episode 48556 reward: -21.0, running: -20.45
Episode 48557 reward: -19.0, running: -20.44
Episode 48558 reward: -20.0, running: -20.43
Episode 48559 reward: -21.0, running: -20.44
Episode 48560 reward: -21.0, running: -20.44
Episode 48561 reward: -21.0, running: -20.45
Episode 48562 reward: -21.0, running: -20.45
Episode 48563 reward: -20.0, running: -20.45
Episode 48564 reward: -21.0, running: -20.46
Episode 48565 reward: -21.0, running: -20.46
Episode 48566 reward: -21.0, running: -20.47
Episode 48567 reward: -21.0, running: -20.47
Episode 48568 reward: -21.0, running: -20.48
Episode 48569 reward: -21.0, running: -20.48
Episode 48570 reward: -21.0, running: -20.49
Episode 48571 reward: -21.0, running: -20.49
Episode 48572 reward: -20.0, running: -20.49
Episode 48

Episode 48734 reward: -20.0, running: -20.46
Episode 48735 reward: -20.0, running: -20.46
Episode 48736 reward: -20.0, running: -20.46
Episode 48737 reward: -21.0, running: -20.46
Episode 48738 reward: -21.0, running: -20.47
Episode 48739 reward: -20.0, running: -20.46
Episode 48740 reward: -21.0, running: -20.47
Episode 48741 reward: -21.0, running: -20.47
Episode 48742 reward: -21.0, running: -20.48
Episode 48743 reward: -21.0, running: -20.48
Episode 48744 reward: -19.0, running: -20.47
Episode 48745 reward: -21.0, running: -20.47
Episode 48746 reward: -21.0, running: -20.48
Episode 48747 reward: -19.0, running: -20.46
Episode 48748 reward: -20.0, running: -20.46
Episode 48749 reward: -21.0, running: -20.46
Episode 48750 reward: -19.0, running: -20.45
Episode 48751 reward: -20.0, running: -20.45
Episode 48752 reward: -19.0, running: -20.43
Episode 48753 reward: -20.0, running: -20.43
Episode 48754 reward: -19.0, running: -20.41
Episode 48755 reward: -21.0, running: -20.42
Episode 48

Episode 48917 reward: -21.0, running: -20.40
Episode 48918 reward: -21.0, running: -20.41
Episode 48919 reward: -20.0, running: -20.40
Episode 48920 reward: -21.0, running: -20.41
Episode 48921 reward: -20.0, running: -20.40
Episode 48922 reward: -21.0, running: -20.41
Episode 48923 reward: -21.0, running: -20.42
Episode 48924 reward: -21.0, running: -20.42
Episode 48925 reward: -20.0, running: -20.42
Episode 48926 reward: -18.0, running: -20.39
Episode 48927 reward: -21.0, running: -20.40
Episode 48928 reward: -21.0, running: -20.41
Episode 48929 reward: -21.0, running: -20.41
Episode 48930 reward: -21.0, running: -20.42
Episode 48931 reward: -21.0, running: -20.42
Episode 48932 reward: -21.0, running: -20.43
Episode 48933 reward: -21.0, running: -20.43
Episode 48934 reward: -21.0, running: -20.44
Episode 48935 reward: -21.0, running: -20.45
Episode 48936 reward: -20.0, running: -20.44
Episode 48937 reward: -21.0, running: -20.45
Episode 48938 reward: -21.0, running: -20.45
Episode 48

Episode 49100 reward: -20.0, running: -20.48
Episode 49101 reward: -21.0, running: -20.48
Episode 49102 reward: -19.0, running: -20.47
Episode 49103 reward: -20.0, running: -20.46
Episode 49104 reward: -19.0, running: -20.45
Episode 49105 reward: -21.0, running: -20.45
Episode 49106 reward: -21.0, running: -20.46
Episode 49107 reward: -21.0, running: -20.47
Episode 49108 reward: -21.0, running: -20.47
Episode 49109 reward: -21.0, running: -20.48
Episode 49110 reward: -21.0, running: -20.48
Episode 49111 reward: -21.0, running: -20.49
Episode 49112 reward: -20.0, running: -20.48
Episode 49113 reward: -20.0, running: -20.48
Episode 49114 reward: -20.0, running: -20.47
Episode 49115 reward: -20.0, running: -20.47
Episode 49116 reward: -21.0, running: -20.47
Episode 49117 reward: -21.0, running: -20.48
Episode 49118 reward: -21.0, running: -20.48
Episode 49119 reward: -20.0, running: -20.48
Episode 49120 reward: -20.0, running: -20.47
Episode 49121 reward: -21.0, running: -20.48
Episode 49

KeyboardInterrupt: 

In [5]:
loss

tensor(-0.0154, grad_fn=<NegBackward0>)